In [ ]:
import os

import numpy
import pandas
import seaborn
import xarray

from src import knn

In [ ]:
seaborn.set()
figsize = (18, 5)

In [ ]:
data = pandas.read_csv('bootstrapped_dates.csv', parse_dates={'date': [0]})
data.set_index('date', inplace=True)

In [ ]:
data

In [ ]:
future_ds_path = os.path.join(knn.DATA_STORE_PATH, 'GCMs/Amazon__pr_day_CanESM5_ssp126_r1i1p1f1_gn_20150101-21001231.nc')

In [ ]:
observed_location = os.path.join(knn.DATA_STORE_PATH, 'OBSERVATIONS/LLdM_AOI2/catalogo.csv')
locations = pandas.read_csv(observed_location)
lon, lat = locations[locations['CODIGO_CAT'] == 'Beni01'][['longitud', 'latitud']].values[0]

In [ ]:
gcm = xarray.open_dataset(future_ds_path)
gcm = knn.shift_longitude_from_360(gcm)
gcm = gcm.sel(lon=lon, lat=lat, method='nearest')

In [ ]:
start_date = data.index.min()
end_date = data.index.max()

In [ ]:
gcm = gcm.sel(time=slice(start_date, end_date))

In [ ]:
gcm_precip = gcm.pr.to_numpy() * (60 * 60 * 24)
data['gcm_precip'] = gcm_precip

In [ ]:
results_r = pandas.read_csv(
    '../test-results/CanESM5/CanESM5_ssp126_pr_1_bootstrap.csv',
    parse_dates={'date': ['simYear', 'simMonth', 'simDay']})
results_r.set_index('date', inplace=True)

In [ ]:
dd = pandas.merge(data, results_r['bs_value'], left_index=True, right_index=True, how='left')

In [ ]:
dd.rename({
    'historic_precip': 'python_prediction',
    'bs_value': 'R_prediction',
    'gcm_precip': 'GCM_prediction'
}, axis=1, inplace=True)

In [ ]:
pal = seaborn.color_palette("deep")
dd[['python_prediction', 'R_prediction', 'GCM_prediction']].plot(
    xlim=('2023-01-01', '2024-07-01'),
    color=[pal[0], pal[1], pal[2]],
    linewidth=1,
    figsize=figsize)

In [ ]:
# seaborn.color_palette('deep')

In [ ]:
dd['python_prediction'].hist(bins=100)

In [ ]:
dd['R_prediction'].hist(bins=100)